<a href="https://colab.research.google.com/github/Swayamprakashpatel/HPLC/blob/main/HPLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dropout
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from google.colab import files
import time as tm

In [11]:
df = pd.read_csv('/content/Data.csv')
df = pd.DataFrame(df)
df = df.iloc[:,:]

X1 = df.iloc[:, 12:14]
X1 = X1.div(20).round(2) 

X2 = df.iloc[:, 14:]
X = [X1,X2]
X = pd.concat(X, axis=1)

Y_Cat = df.iloc[:, 4:7]
Y_Num = df.iloc[:,7:11]
Y_Num = Y_Num.div(100).round(2)

X.head(),Y_Cat.head(), Y_Num.head()
print(X.shape, Y_Cat.shape, Y_Num.shape)

(1072, 1764) (1072, 3) (1072, 4)


In [16]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
print(X)

CLASSIFICATION

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(Y_Cat)
OneHotEncoder(handle_unknown='ignore')
enc.categories_

Y_Cat_OH = enc.transform(Y_Cat).toarray() #Categorical Data
print(Y_Cat_OH,Y_Cat_OH.shape)

print(Y_Cat_OH.shape)



Y2 = Y_Cat_OH

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X_train, X_val_and_test, Y2_train, Y2_val_and_test = train_test_split(X, Y2, test_size=0.01,random_state = 42 )
X_val, X_test, Y2_val, Y2_test = train_test_split(X_val_and_test, Y2_val_and_test, test_size=0.5, random_state= 42)

X_train = np.asarray(X_train).astype(np.int64)
X_val = np.asarray(X_val).astype(np.int64)
X_test = np.asarray(X_test).astype(np.int64)
Y2_train = np.asarray(Y2_train).astype(np.int64)
Y2_val = np.asarray(Y2_val).astype(np.int64)
Y2_test = np.asarray(Y2_test).astype(np.int64)


filepath = '/content/drive/MyDrive/Model_DE/Model.hdf5'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor='accuracy', mode='max', save_best_only=True, Save_weights_only = False, verbose = 1), 
              tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=225, verbose =2)]

output_nodes = Y2.shape[1]
print(output_nodes)
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(1764, activation='relu', input_shape=(1764,)),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(output_nodes, activation= 'Softmax')])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.8), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics=['accuracy'])
hist = model.fit(X_train, Y2_train, epochs= 2000, callbacks=[checkpoint],validation_data=(X_val, Y2_val), batch_size= 150)
model.evaluate(X_test, Y2_test)
 
Y2_train_predict = np.round(model.predict(X_train))
Y2_train_l = tf.argmax(Y2_train, axis = 1)
Y2_train_predict_l = tf.argmax(Y2_train_predict, axis =1)
import sklearn.metrics as skm
cm = skm.multilabel_confusion_matrix(Y2_train_l, Y2_train_predict_l)
print(cm)
print( skm.classification_report(Y2_train_l, Y2_train_predict_l))
 
train_acc = max(hist.history['accuracy'])
val_acc = max(hist.history['val_accuracy'])
train_loss = min(hist.history['loss'])
val_loss = min(hist.history['val_loss'])
print('Training Accuracy is')
print(train_acc)
print('Validation Accuracy is')
print(val_acc)
print('Training loss is')
print(train_loss)
print('Validation loss is')
print(val_loss)

In [7]:
X.isnull().any() 


RT_Drug_1    False
RT_Drug_2    False
fp1          False
fp2          False
fp3          False
             ...  
fp877.1      False
fp878.1      False
fp879.1      False
fp880.1      False
fp881.1      False
Length: 1764, dtype: bool

REGRESSION

In [13]:
Y = Y_Num


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.3,random_state = 42 )
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5, random_state= 42)
import numpy as np
 
X_train = np.asarray(X_train).astype(np.int64)
X_val = np.asarray(X_val).astype(np.int64)
X_test = np.asarray(X_test).astype(np.int64)
Y_train = np.asarray(Y_train).astype(np.int64)
Y_val = np.asarray(Y_val).astype(np.int64)
Y_test = np.asarray(Y_test).astype(np.int64)
 
filepath = '/content/drive/My Drive/SOLModel.hdf5'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_root_mean_squared_error', mode='min', save_best_only=True, Save_weights_only = False, verbose = 2), 
              tf.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', patience=575, verbose =2)]
output_nodes = Y.shape[1]
print(output_nodes)


model = tf.keras.Sequential([
                             tf.keras.layers.Dense(512, activation='relu', input_shape=(1764,)),
                             tf.keras.layers.Dense(512, activation='selu'),
                             #tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(output_nodes, activation ='Softmax')])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate = 0.7), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
hist = model.fit(X, Y, epochs= 10000, callbacks=[checkpoint],validation_data=(X, Y), batch_size= 100)
model.evaluate(X_test, Y_test)
 
# Error Graph

plt.plot(hist.history['root_mean_squared_error'])
plt.plot(hist.history['val_root_mean_squared_error'])
plt.title('Model RMSE')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()
 

train_acc = min(hist.history['root_mean_squared_error'])
val_acc = min(hist.history['val_root_mean_squared_error'])
train_loss = min(hist.history['loss'])
val_loss = min(hist.history['val_loss'])
print('Training RMSE is')
print(train_acc)
print('Validation RMSE is')
print(val_acc)
print('Training loss is')
print(train_loss)
print('Validation loss is')
print(val_loss)



Streaming output truncated to the last 5000 lines.
Epoch 8751: val_root_mean_squared_error did not improve from 0.08840
11/11 [==============================] - 0s 41ms/step - loss: 0.0108 - root_mean_squared_error: 0.1038 - val_loss: 0.0091 - val_root_mean_squared_error: 0.0956
Epoch 8752/10000
11/11 [==============================] - ETA: 0s - loss: 0.0105 - root_mean_squared_error: 0.1023
Epoch 8752: val_root_mean_squared_error did not improve from 0.08840
11/11 [==============================] - 0s 42ms/step - loss: 0.0105 - root_mean_squared_error: 0.1023 - val_loss: 0.0083 - val_root_mean_squared_error: 0.0911
Epoch 8753/10000
10/11 [==========================>...] - ETA: 0s - loss: 0.0096 - root_mean_squared_error: 0.0979
Epoch 8753: val_root_mean_squared_error did not improve from 0.08840
11/11 [==============================] - 0s 42ms/step - loss: 0.0098 - root_mean_squared_error: 0.0988 - val_loss: 0.0125 - val_root_mean_squared_error: 0.1116
Epoch 8754/10000
10/11 [========

NameError: ignored

In [13]:
print(Y)

      %METHANOL  %ACETONITRILE  %WATER
0          0.00           0.15     0.0
1          0.40           0.00     0.0
2          0.00           0.15     0.0
3          0.60           0.00     0.0
4          0.00           0.60     0.0
...         ...            ...     ...
1098       0.30           0.30     0.0
1099       0.20           0.50     0.0
1100       0.00           0.00     0.9
1101       0.00           0.80     0.2
1102       0.75           0.00     0.0

[1103 rows x 3 columns]


In [ ]:
t1 = model.predict(X_train[0:5,:])
print(t1)
 
t = np.round(model.predict(X_train))
print(t)
 
Y_prediction = enc.inverse_transform(t)
print(Y1.head(),Y_prediction, Y_prediction.shape)


[[1.71530455e-01 8.16830218e-01 6.65873110e-01 3.60201508e-01
  1.99705362e-04 2.29805708e-04 6.58282876e-01 3.73521060e-01
  2.51263380e-04 2.94816494e-03 1.98728012e-05 8.19228371e-05
  3.87079358e-01 4.11924448e-05 2.22253799e-03 4.13147867e-01
  2.75138021e-03 2.40655124e-01 8.58038664e-04 1.06510520e-03
  6.77317381e-04 7.55854368e-01 2.60028243e-03 2.05155629e-05
  2.05650061e-01 1.74760818e-04 9.11428579e-05 9.99983191e-01
  7.51295805e-01 2.40557075e-01 8.49283970e-05 2.69323587e-04
  1.92046165e-04 1.62856122e-05 1.26387904e-05]
 [4.54753081e-06 9.99998808e-01 2.39379663e-08 9.99999285e-01
  1.53970058e-21 4.55224325e-17 9.99996603e-01 1.51708662e-06
  5.24412413e-15 5.02116237e-10 1.25044635e-20 1.46101262e-18
  9.99990106e-01 7.79245744e-18 7.07244983e-12 3.66711191e-07
  1.82847085e-10 8.12716561e-09 7.39361304e-14 1.21551515e-14
  1.19079825e-18 2.98875129e-05 3.23215318e-05 9.99941826e-01
  7.37296613e-11 6.56344163e-12 6.15678307e-15 1.00000000e+00
  1.00000000e+00 1.538

In [ ]:
X.to_csv('Drug_1.csv')